# Lesson 4: Auto-merging Retrieval

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 269ccce0-440e-4785-a368-06702ec6b531
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Auto-merging retrieval setup

In [5]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
from llama_index.node_parser import HierarchicalNodeParser

# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
nodes = node_parser.get_nodes_from_documents([document])

In [8]:
from llama_index.node_parser import get_leaf_nodes

leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[30].text)

But this became less important as numerical linear algebra libraries matured.
Deep learning is still an emerging technology, so when you train a neural network and the 
optimization algorithm struggles to converge, understanding the math behind gradient 
descent, momentum, and the Adam  optimization algorithm will help you make better decisions. 
Similarly, if your neural network does something funny — say, it makes bad predictions on 
images of a certain resolution, but not others — understanding the math behind neural network 
architectures puts you in a better position to figure out what to do.
Of course, I also encourage learning driven by curiosity.


In [9]:
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
print(parent_node.text)

PAGE 12Should You 
Learn Math to 
Get a Job in AI? CHAPTER 3
LEARNING

PAGE 13Should you Learn Math to Get a Job in AI? CHAPTER 3
Is math a foundational skill for AI? It’s always nice to know more math! But there’s so much to 
learn that, realistically, it’s necessary to prioritize. Here’s how you might go about strengthening 
your math background.
To figure out what’s important to know, I find it useful to ask what you need to know to make 
the decisions required for the work you want to do. At DeepLearning.AI, we frequently ask, 
“What does someone need to know to accomplish their goals?” The goal might be building a 
machine learning model, architecting a system, or passing a job interview.
Understanding the math behind algorithms you use is often helpful, since it enables you to 
debug them. But the depth of knowledge that’s useful changes over time. As machine learning 
techniques mature and become more reliable and turnkey, they require less debugging, and a 
shallower understand

### Building the index

In [10]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [11]:
from llama_index import ServiceContext

auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser,
)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [12]:
from llama_index import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

automerging_index.storage_context.persist(persist_dir="./merging_index")

In [ ]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./merging_index"):
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
            service_context=auto_merging_context
        )

    automerging_index.storage_context.persist(persist_dir="./merging_index")
else:
    automerging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./merging_index"),
        service_context=auto_merging_context
    )


### Defining the retriever and running the query engine

In [13]:
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=12
)

retriever = AutoMergingRetriever(
    automerging_retriever, 
    automerging_index.storage_context, 
    verbose=True
)

rerank = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")

auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank]
)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [14]:
auto_merging_response = auto_merging_engine.query(
    "What is the importance of networking in AI?"
)

In [15]:
from llama_index.response.notebook_utils import display_response

display_response(auto_merging_response)

**`Final Response:`** Networking in AI is crucial as it helps individuals build a strong professional community that can provide valuable information, support, and opportunities. By connecting with others in the field, individuals can receive help, advice, and referrals to potential employers. Additionally, networking can lead to collaborations, idea-sharing, and the chance to learn from domain experts, ultimately helping individuals advance their skills and expertise in AI.

## Putting it all Together

In [16]:
import os

from llama_index import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.node_parser import HierarchicalNodeParser
from llama_index.node_parser import get_leaf_nodes
from llama_index import StorageContext, load_index_from_storage
from llama_index.retrievers import AutoMergingRetriever
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [17]:
from llama_index.llms import OpenAI

index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./merging_index",
)


In [18]:
query_engine = get_automerging_query_engine(index, similarity_top_k=6)

## TruLens Evaluation

In [19]:
from trulens_eval import Tru

Tru().reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### Two layers

In [20]:
auto_merging_index_0 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048,512],
)

In [21]:
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [22]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_0,
    app_id ='app_0'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [23]:
eval_questions = []
with open('generated_questions.text', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [24]:
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [25]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_0)

> Merging 1 nodes into parent node.
> Parent node id: 5f46b2cb-4928-471d-8d98-1695bcc98fd5.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 1 nodes into parent node.
> Parent node id: d8f9d338-afd4-42e7-9f75-b048d0decb03.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...

> Merging 1 nodes into parent node.
> Parent node id: 81188c09-885c-4baf-b5b9-7df6db6e6aa0.
> Parent node text: PAGE 33Choose who to work with. It’s tempting to take a position because of the projects you’ll w...

> Merging 1 nodes into parent node.
> Parent node id: f479ff09-3a32-499c-b913-a353e06540a3.
> Parent node text: PAGE 23Each project is only one step on a longer journey, hopefully one that has a positive impac...

> Merging 1 nodes into parent node.
> Parent node id: 11b98656-e2d3-4340-a4b9-5dd307f583be.
> Parent node text: PAGE 29If you’re preparing to sw

> Merging 1 nodes into parent node.
> Parent node id: ac58dc56-903e-458b-b841-d807e237e0b7.
> Parent node text: PAGE 13Should you Learn Math to Get a Job in AI? CHAPTER 3
Is math a foundational skill for AI? I...

> Merging 2 nodes into parent node.
> Parent node id: 53e2e153-5aab-4c22-965e-a191a17bfcaa.
> Parent node text: PAGE 9In the previous chapter, I introduced three key steps for building a career in AI: learning...

> Merging 1 nodes into parent node.
> Parent node id: 6aba5314-5c0c-48f4-a0a0-c181c2a50d19.
> Parent node text: PAGE 4Coding AI Is the New Literacy
Today we take it for granted that many people know how to rea...

> Merging 1 nodes into parent node.
> Parent node id: bcb32dc8-c357-4edc-838c-c15c9f3e5160.
> Parent node text: PAGE 10This is a lot to learn!
Even after you master everything on this list, I hope you’ll keep ...

> Merging 1 nodes into parent node.
> Parent node id: 2fba6f6c-a395-4019-ba5d-47ed6dec138c.
> Parent node text: PAGE 18It goes without saying th

> Merging 1 nodes into parent node.
> Parent node id: 11b98656-e2d3-4340-a4b9-5dd307f583be.
> Parent node text: PAGE 29If you’re preparing to switch roles (say, taking a job as a machine learning engineer for ...

> Merging 1 nodes into parent node.
> Parent node id: b73bcb39-f4b2-4283-a8b7-cb80cd2ff606.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: ca423a10-8fcf-4b27-96ef-ee326d0ad842.
> Parent node text: PAGE 7These phases apply in a wide 
range of professions, but AI 
involves unique elements.
For e...

> Merging 1 nodes into parent node.
> Parent node id: 2fba6f6c-a395-4019-ba5d-47ed6dec138c.
> Parent node text: PAGE 18It goes without saying that we should only work on projects that are responsible, ethical,...

> Merging 1 nodes into parent node.
> Parent node id: 81188c09-885c-4baf-b5b9-7df6db6e6aa0.
> Parent node text: PAGE 33Choose who to work with. 

> Merging 1 nodes into parent node.
> Parent node id: d8ec0c88-cebe-4b60-9c9a-0571e72cf156.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: 94486094-552b-4c85-b2a2-4f74afa9c427.
> Parent node text: PAGE 39My three-year-old daughter (who can barely count to 12) regularly tries to teach things to...

> Merging 1 nodes into parent node.
> Parent node id: 74b6cf64-4a87-432e-85ae-66cc15ff357d.
> Parent node text: PAGE 37Overcoming Imposter 
SyndromeCHAPTER 11

> Merging 1 nodes into parent node.
> Parent node id: b73bcb39-f4b2-4283-a8b7-cb80cd2ff606.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: f8322aa9-9603-4a5f-9292-29757469fb48.
> Parent node text: PAGE 35Keys to Building a Career in AI CHAPTER 10
The path to career success in AI is 

> Merging 1 nodes into parent node.
> Parent node id: 2fba6f6c-a395-4019-ba5d-47ed6dec138c.
> Parent node text: PAGE 18It goes without saying that we should only work on projects that are responsible, ethical,...

> Merging 1 nodes into parent node.
> Parent node id: 218005a5-29e4-4571-8675-e3fcd511f8e3.
> Parent node text: PAGE 16Determine milestones. Once you’ve deemed a project sufficiently 
valuable, the next step i...

> Merging 1 nodes into parent node.
> Parent node id: 5a453f42-c4e3-45eb-bd32-1f33db8d8fe3.
> Parent node text: PAGE 22Over the course of a career, you’re likely to work on projects in succession, each growing...

> Merging 1 nodes into parent node.
> Parent node id: b73bcb39-f4b2-4283-a8b7-cb80cd2ff606.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: ca423a10-8fcf-4b27-96ef-ee326d0ad842.
> Parent node text: PAGE 7These phases apply in a wi

> Merging 1 nodes into parent node.
> Parent node id: b73bcb39-f4b2-4283-a8b7-cb80cd2ff606.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 94486094-552b-4c85-b2a2-4f74afa9c427.
> Parent node text: PAGE 39My three-year-old daughter (who can barely count to 12) regularly tries to teach things to...

> Merging 1 nodes into parent node.
> Parent node id: 4e01cba7-8ea6-4c51-9885-4a67e49d1f6b.
> Parent node text: PAGE 6The rapid rise of AI has led to a rapid rise in AI jobs, and many people are building excit...

> Merging 1 nodes into parent node.
> Parent node id: a78c25e8-e4d3-4080-b8de-8c4c6a71b875.
> Parent node text: PAGE 36Keys to Building a Career in AI CHAPTER 10
Of all the steps in building a career, this 
on...

> Merging 1 nodes into parent node.
> Parent node id: f8322aa9-9603-4a5f-9292-29757469fb48.
> Parent node text: PAGE 35Keys to Building a Career

In [26]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
app_0,0.834921,0.277778,0.988889,3.875,0.003671


In [ ]:
Tru().run_dashboard()

### Three layers

In [33]:
auto_merging_index_1 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_1",
    chunk_sizes=[2048,512,128],
)

In [34]:
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=12,
    rerank_top_n=6,
)


In [35]:
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_1,
    app_id ='app_1'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [36]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_1)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f86a0417f10 is calling an instrumented method <function BaseQueryEngine.query at 0x7f88fa924160>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f87e41a08b0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f86a0417f10 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f88f6343910>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f87e41a08b0) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x7f874c1429e0 is calling an instrumented method <function BaseRetriever.retrieve at 0x7f88fa903490>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7

> Merging 4 nodes into parent node.
> Parent node id: 418aaa73-5a57-4c19-8307-a7955b107ab9.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 5 nodes into parent node.
> Parent node id: 7aabb0e8-8343-4174-9225-d76cabfb5152.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...

> Merging 1 nodes into parent node.
> Parent node id: da17878b-4854-42d5-8cca-edc3b270bbc6.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 1 nodes into parent node.
> Parent node id: efb5968d-c9e1-4e22-b71a-1e8ec20a3d50.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f86a0417e80 is calling an instrumented method <function CompactAndRefine.get_response at 0x7f88f9e4c040>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f87e41a0820) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f86a0417e80 is calling an instrumented method <function Refine.get_response at 0x7f88f935b9a0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f87e41a0820) using this function.
A new object of type <class 'llama_index.llm_predictor.base.LLMPredictor'> at 0x7f87682d8440 is calling an instrumented method <function LLMPredictor.predict at 0x7f8906fcdbd0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthes

> Merging 5 nodes into parent node.
> Parent node id: 7fe8ceb5-7d06-41c3-a5d0-3c1862f40c65.
> Parent node text: PAGE 27There’s a lot we don’t know about the future: When will we cure Alzheimer’s disease? Who w...

> Merging 1 nodes into parent node.
> Parent node id: ea2ccfe8-4123-4eea-834f-3fb56e3d4a74.
> Parent node text: PAGE 27There’s a lot we don’t know about the future: When will we cure Alzheimer’s disease? Who w...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f86a0417e80 is calling an instrumented method <function Refine.get_response at 0x7f88f935b9a0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f87e41a0820) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f86a0417f10 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f88f6343910>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f87e41a08b0) using this function.


...

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f86a0417e80 is calling an instrumented method <function Refine.get_response at 0x7f88f935b9a0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f87e41a0820) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f86a0417f10 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f88f6343910>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f87e41a08b0) using this function.


> Merging 2 nodes into parent node.
> Parent node id: ee1bd2e9-0a7f-4c43-8fc2-cbedf40d297d.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 82abe983-c16d-45f9-add5-fde3606a36de.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f86a0417e80 is calling an instrumented method <function Refine.get_response at 0x7f88f935b9a0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f87e41a0820) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f86a0417f10 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f88f6343910>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f87e41a08b0) using this function.


> Merging 4 nodes into parent node.
> Parent node id: 91a81465-a71b-444f-8be3-c164581c755e.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 3 nodes into parent node.
> Parent node id: 8512b841-812f-459d-8043-9bac2559d282.
> Parent node text: PAGE 39My three-year-old daughter (who can barely count to 12) regularly tries to teach things to...

> Merging 1 nodes into parent node.
> Parent node id: 02c8f6ba-37ab-40aa-9875-d24c70e63048.
> Parent node text: PAGE 37Overcoming Imposter 
SyndromeCHAPTER 11

> Merging 1 nodes into parent node.
> Parent node id: 09d3134e-6c94-4c86-9ec4-0600b77b8f9d.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: bbb8c8fd-9078-4fca-a7e2-73446a9b0389.
> Parent node text: PAGE 39My three-year-old daughter (who can barely count to 12) regularly tries to teac

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f86a0417e80 is calling an instrumented method <function Refine.get_response at 0x7f88f935b9a0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f87e41a0820) using this function.


In [37]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
app_0,0.708132,0.451282,0.991667,3.8750,0.003671
app_1,NaN,NaN,0.980000,3.3125,0.001888


In [38]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-113-66p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>